## Results 1 - Hate Speech Detection

Let's check where the context really helps

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [5]:
%load_ext autoreload
%autoreload 2

from hatedetection import load_datasets
import glob
import json

train_dataset, dev_dataset, test_dataset = load_datasets(add_body=True)

no_context_evals = []
context_evals = []

for path in glob.glob("../evaluations/non-context*"):
    if "category" in path:
        continue
    with open(path) as f:
        obj = json.load(f)
        obj["file"] = path
        no_context_evals.append(obj)

for path in glob.glob("../evaluations/context*"):
    if "category" in path:
        continue

    with open(path) as f:
        obj = json.load(f)
        obj["file"] = path
        context_evals.append(obj)

print(f"We have {len(context_evals)} context evaluations")
print(f"We have {len(no_context_evals)} context evaluations")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
We have 10 context evaluations
We have 10 context evaluations


In [6]:
import pandas as pd

df_context_evals = pd.DataFrame([
    {**{"file": evaluation["file"]}, **evaluation["metrics"]} for evaluation in context_evals
])

df_context_evals

,file,eval_loss,eval_accuracy,eval_f1,eval_macro_f1,eval_precision,eval_recall,eval_runtime,eval_samples_per_second,init_mem_cpu_alloc_delta,init_mem_gpu_alloc_delta,init_mem_cpu_peaked_delta,init_mem_gpu_peaked_delta,test_mem_cpu_alloc_delta,test_mem_gpu_alloc_delta,test_mem_cpu_peaked_delta,test_mem_gpu_peaked_delta
0,../evaluations/context-6.json,0.813584,0.906109,0.670788,0.808018,0.754520,0.603784,85.5273,132.624,51854,0,18258,0,405165,0,887161,189087744
1,../evaluations/context-3.json,0.840038,0.903905,0.665644,0.804767,0.741627,0.603784,172.4795,65.764,51854,0,18258,0,409133,0,887288,189087744
2,../evaluations/context-4.json,0.807908,0.910253,0.682668,0.815202,0.776045,0.609349,85.0441,133.378,51782,0,18258,0,405647,0,887233,189087744
3,../evaluations/context-7.json,0.795504,0.907696,0.681279,0.813656,0.752016,0.622705,89.3854,126.900,51782,0,18258,0,405138,0,887161,189087744
4,../evaluations/context-1.json,0.802624,0.905052,0.665632,0.805151,0.752809,0.596550,219.5260,51.670,51854,0,18258,0,405281,0,887225,189087744
5,../evaluations/context-2.json,0.812863,0.905757,0.667703,0.806398,0.756338,0.597663,217.8945,52.057,51854,0,18258,0,405778,0,887161,189087744
6,../evaluations/context-9.json,0.845237,0.902407,0.657594,0.800344,0.740251,0.591541,84.4517,134.314,51854,0,18258,0,405583,0,887161,189087744
7,../evaluations/context-10.json,0.806833,0.901878,0.664456,0.803496,0.725000,0.613244,84.9726,133.490,51854,0,18258,0,405599,0,887224,189087744
8,../evaluations/context-8.json,0.773366,0.904082,0.664198,0.804124,0.745669,0.598776,90.7453,124.998,51854,0,18258,0,405282,0,887161,189087744
9,../evaluations/context-5.json,0.569686,0.906021,0.673407,0.809260,0.749148,0.611575,84.7567,133.830,51854,0,18258,0,405147,0,887224,189087744


In [9]:
metric_columns = ["eval_precision", "eval_recall", "eval_f1", "eval_macro_f1"]

context_cat_df = pd.DataFrame({"context mean": df_context_evals[metric_columns].mean(), "context std": df_context_evals[metric_columns].std()})

df_no_context_evals = pd.DataFrame([
    {**{"file": evaluation["file"]}, **evaluation["metrics"]} for evaluation in no_context_evals
])

no_context_cat_df = pd.DataFrame({
    "no context mean": df_no_context_evals[metric_columns].mean(), 
    "no context std": df_no_context_evals[metric_columns].std()})


result_df = pd.concat([context_cat_df, no_context_cat_df], axis=1)

result_df

,context mean,context std,no context mean,no context std
eval_precision,0.749342,0.013150,0.676895,0.018041
eval_recall,0.604897,0.009378,0.597385,0.015157
eval_f1,0.669337,0.007868,0.634311,0.005115
eval_macro_f1,0.807042,0.004611,0.785090,0.003015


Hay que chequear qué pasa con los FP de no usar contexto